# Lesson 02 - CRAM Basics

## Loading the tutorial package

`ros-load:load-system` loads an ASDF (Another System Definition Facility) syste, from within a Catkin package.

In [4]:
(asdf:load-system :cram-pr2-pick-place-demo)

T

In [5]:
(defpackage cram-bullet-world-tutorial
  (:nicknames #:btw-tut)
   (:use #:common-lisp #:cram-prolog
        #:desig #:exe)
  (:export))

#<PACKAGE "CRAM-BULLET-WORLD-TUTORIAL">

In [7]:
(in-package :btw-tut)

#<PACKAGE "CRAM-BULLET-WORLD-TUTORIAL">

Now to launch a simulation with a kitchen environment and our robot, type the following command into your Lisp command line:

In [8]:
(roslisp-utilities:startup-ros)

NIL

[(ROSLISP TOP) INFO] 1663438498.716: Node name is /cram_hl_691166_1663438498
[(ROSLISP TOP) INFO] 1663438498.717: Namespace is /
[(ROSLISP TOP) INFO] 1663438498.717: Params are NIL
[(ROSLISP TOP) INFO] 1663438498.717: Remappings are:
[(ROSLISP TOP) INFO] 1663438498.717: master URI is 127.0.0.1:11311
[(ROSLISP TOP) INFO] 1663438501.731: Node startup complete
[(ROSNODE) INFO] 1663438501.737: ROS init #<FUNCTION CRAM-TF::INIT-TF>.
[(CRAM-TF INIT-TF) INFO] 1663438501.737: Set *fixed-frame* to "map".
[(CRAM-TF INIT-TF) INFO] 1663438501.848: Environment name is IAI-KITCHEN.
[(CRAM-TF INIT-TF) INFO] 1663438501.879: Initialized *transformer* to a TRANSFORM-LISTENER.
[(CRAM-TF INIT-TF) INFO] 1663438501.886: *tf-default-timeout* is 4.0.
[(CRAM-TF INIT-TF) INFO] 1663438501.906: Robot name is PR2.
[(CRAM-TF INIT-TF) INFO] 1663438501.911: Set *ODOM-FRAME* to "odom_combined".
[(CRAM-TF INIT-TF) INFO] 1663438501.911: Set *ROBOT-BASE-FRAME* to "base_footprint".
[(CRAM-TF INIT-TF) INFO] 1663438501.911:

In [10]:
(demo::init-projection)

(:BLUE-METAL-PLATE :BOTTLE :BOWL :BREAKFAST-CEREAL :BUTTERMILK :CEREAL :CUBE
 :CUP :FORK :KNIFE :MILK :MILK :RED-METAL-PLATE :SPOON :SPOON :TRAY-BASE :TRAY)

[(SET-ROBOT-STATE-FROM-TF) WARN] 1663438539.241: Failed with transform-stamped-error:
    No transform was published between frames base_footprint and map
    Ignore this warning if no real robot is running.


In [11]:
(btr-utils:reset-debug-window)

NIL

## Moving Around



In [12]:
(defun make-pose (reference-frame pose)
    "Creates a cl-transforms pose in 3D space w.r.t. the given `reference-frame' and `pose'.
    The pose is given as a list of two lists, where the first are the x y z coordinates, 
    the second is the quaternion of qx qy qz w, like this: '((2.0 0.0 1.5) (0.0 0.0 0.0 1.0))"
  (cl-transforms-stamped:make-pose-stamped
   reference-frame 0.0
   (apply #'cl-transforms:make-3d-vector (first pose))
   (apply #'cl-transforms:make-quaternion (second pose))))

MAKE-POSE

In [13]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal (make-pose "map" '((0 1 0) (0 0 0 1)))))
    (perform (an action
                 (type going)
                 (target (a location 
                            (pose ?navigation-goal)))))))

#<HASH-TABLE :TEST EQUAL :COUNT 90 {100443F003}>

In [18]:
(defparameter *base-pose-near-table*
    "A pose for the robot to stand near the table"
  (make-pose "map" 
             '((-2.2d0 -0.20d0 0.0d0) 
               (0.0d0 0.0d0 -0.707d0 0.707d0))))

*BASE-POSE-NEAR-TABLE*

In [22]:
(urdf-proj:with-simulated-robot
  (let ((?navigation-goal *base-pose-near-table*))
     (perform (an action
                 (type going)
                  (target (a location 
                             (pose ?navigation-goal)))))))

#<HASH-TABLE :TEST EQUAL :COUNT 90 {1011DC98D3}>

Advanced definition of poses with euler angles provides a quaternion.

In [20]:
(cl-transforms:euler->quaternion :az (* pi 0.5)) ;; rotation of pi/2 around the z-axis = 90 degree to the left

#<CL-TRANSFORMS:QUATERNION (0.0d0 0.0d0 0.7071067811865475d0 0.7071067811865476d0)>

The resulting quaternion can be used to specify rotations around an axis. Here we a creating a pose without the help of the `make-pose` function.

In [21]:
(defparameter *base-pose-near-table*
  (cl-transforms-stamped:make-pose-stamped 
   "map" 0.0 
   (cl-transforms:make-3d-vector -2.2d0 -0.20d0 0.0d0)
   (cl-transforms:euler->quaternion :az (* pi -0.5))))

*BASE-POSE-NEAR-TABLE*

## Spawning Objects

In [31]:
(defun spawn-object (spawn-pose &optional (obj-type :bottle) (obj-name 'bottle-1) (obj-color '(1 0 0)))
    "Spawns an object into simulation.
    spawn-pose: list - target pose coordinates like this: '((x y z) (qx qy qz w))
    optional:
      obj-type: keyword - the type of object to spawn, like :bottle (case-insensitive)
      obj-name: symbol - name of the object, like 'bottle-1 (case-insensitive)
      obj-color: list - (r g b) color value for the object. Range per value is 0.0 - 1.0"
  (unless (assoc obj-type btr::*mesh-files*)
    (btr:add-objects-to-mesh-list "cram_pr2_pick_place_demo"))
  (btr-utils:spawn-object obj-name obj-type :color obj-color :pose spawn-pose)
  (btr:simulate btr:*current-bullet-world* 10))

SPAWN-OBJECT

SB-KERNEL:REDEFINITION-WITH-DEFUN: redefining CRAM-BULLET-WORLD-TUTORIAL::SPAWN-OBJECT in DEFUN


To find all possible `obj-type`, the parameter `btr::*mesh-files*` contains the mesh files as key-value pairs between keyword and the location of their mesh-file. We can map over that list and only return their keys. 

In [35]:
;; the parameter btr::*mesh-files* contains the mesh files 
(map 'list #'first btr::*mesh-files*)

(:TRAY-BASE :SPOON :RED-METAL-PLATE :MILK :CUBE :CEREAL :BUTTERMILK
 :BREAKFAST-CEREAL :BOTTLE :BLUE-METAL-PLATE :MUG :MUG-COMPOUND :PLATE
 :PLATE-COMPOUND :TRAY :TRAY-COMPOUND :CUP-NON-COMPOUND :CUP :MONDAMIN :POT
 :WEISSWURST :BOWL-ORIGINAL :BOWL-NON-COMPOUND :BOWL :FORK :KNIFE :SPATULA :CAP
 :GLASSES :GLOVE :SHOE :ARROW)

## Poses and stuff

In [3]:

   
;;; ****Coordinate Visualization 

(defun visualize-coordinates (object-or-pose &optional (size 0.3))
  (flet ((find-object-of-type (type)
           (find type
                 (remove-if-not (lambda (obj) (typep obj 'btr:item))
                                (btr:objects btr:*current-bullet-world*))
                 :test (lambda (type item) (eql type (car (slot-value item 'btr::types)))))))
    (etypecase object-or-pose
      (symbol
       (if (btr:object btr:*current-bullet-world* object-or-pose)
           (btr:add-vis-axis-object object-or-pose :length size)
           (if (find-object-of-type object-or-pose)
               (btr:add-vis-axis-object
                (btr:name (find-object-of-type object-or-pose))
                :length size)
               (warn "Unknown object, please either give an object name or object type."))))
      (cl-transforms-stamped:pose-stamped
       (when (equalp (cl-transforms-stamped:frame-id object-or-pose) "base_footprint")
         (warn "Pose is not in MAP frame. It is visualized with respect to the MAP though."))
       (btr:add-vis-axis-object object-or-pose :length size))
      (cl-transforms:pose
       (btr:add-vis-axis-object object-or-pose :length size)))))



(defun move-kitchen-joint (&key (joint-name "iai_fridge_door_joint")
                             (joint-angle 0.2d0))
  (btr:set-robot-state-from-joints
   `((,joint-name  ,joint-angle))
   (btr:get-environment-object)))

(defun add-objects-to-mesh-list (&optional (ros-package "cram_bullet_world_tutorial"))
  (mapcar (lambda (object-filename-and-object-extension)
            (declare (type list object-filename-and-object-extension))
            (destructuring-bind (object-filename object-extension)
                object-filename-and-object-extension
              (let ((lisp-name (roslisp-utilities:lispify-ros-underscore-name
                                object-filename :keyword)))
                (pushnew (list lisp-name
                               (format nil "package://~a/resource/~a.~a"
                                       ros-package object-filename object-extension)
                               nil)
                         btr::*mesh-files*
                         :key #'car)
                lisp-name)))
          (mapcar (lambda (pathname)
                    (list (pathname-name pathname) (pathname-type pathname)))
                  (directory (physics-utils:parse-uri
                              (format nil "package://~a/resource/*.*" ros-package))))))
                              
;;; ****Define Object 
(defun spawn-object (spawn-pose &optional (obj-type :bottle) (obj-name 'bottle-1) (obj-color '(1 0 0)))
  (unless (assoc obj-type btr::*mesh-files*)
    (btr:add-objects-to-mesh-list "cram_pr2_pick_place_demo"))
  (btr-utils:spawn-object obj-name obj-type :color obj-color :pose spawn-pose)
  (btr:simulate btr:*current-bullet-world* 10))
;;; ****List of Object
(defun list-available-objects ()
  (mapcar #'car btr::*mesh-files*))
;;;; **** ARM Parking
(defun park-arms ()
  (pp-plans::park-arms))

(defun park-arm (arm)
  (pp-plans::park-arms :arm arm))

;;**** Failure Handler
(defmacro handle-failure (errors program-body &body error-handler-body)
  `(progn
    (cpl:with-failure-handling
        ((,errors (e)
           (print e)
           ,@error-handler-body))
    ,@program-body)))


(defparameter *pose-bottle-1*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -2 -0.9d0 0.86d0)
   (cl-transforms:make-identity-rotation)))

(defparameter *pose-bottle-2*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -0.8 2 0.9)
   (cl-transforms:make-identity-rotation)))

(defparameter *pose-meal-table*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -0.15 2.0 0)
   (cl-transforms:make-quaternion 0.0d0 0.0d0 -1.0d0 0.0d0)))

(defparameter *pose-counter*
  (cl-transforms-stamped:make-pose-stamped
   "map" 0.0
   (cl-transforms:make-3d-vector -1.8547d0 -0.381d0 0.0d0)
   (cl-transforms:axis-angle->quaternion (cl-transforms:make-3d-vector 0 0 1) (/ pi -2))))

(defun spawn-two-bottles ()
  (unless (assoc :bottle btr::*mesh-files*)
    (add-objects-to-mesh-list))
  (prolog:prolog
   `(and (btr:bullet-world ?world)
         (assert (btr:object ?world :mesh bottle-1 ((-2 -0.9 0.860) (0 0 0 1))
                             :mass 0.2 :color (1 0 0) :mesh :bottle))
         (assert (btr:object ?world :mesh bottle-2 ((-0.8 2 0.9) (0 0 0 1))
                             :mass 0.2 :color (0 1 0) :mesh :bottle))
         (btr:simulate ?world 100))))

(defun navigate-to (?navigation-goal)
  (exe:perform (desig:a motion
                        (type going)
                        (pose ?navigation-goal))))

(defun look-at (?point-of-interest)
  (exe:perform (desig:a motion
                        (type looking)
                        (pose ?point-of-interest))))

(defun get-perceived-bottle-desig ()
  (let* ((?bottle-desig (desig:an object (type bottle)))
         (?perceived-bottle-desig (exe:perform
                                   (desig:a motion
                                            (type detecting)
                                            (object ?bottle-desig)))))
    ?perceived-bottle-desig))

(defun pick-up (?object-designator &optional (?arm :right))
  (exe:perform (desig:an action
                         (type picking-up)
                         (arm ?arm)
                         (object ?object-designator))))

(defun place-down (?pose ?object ?arm)
  (exe:perform (desig:an action
                         (type placing)
                         (arm ?arm)
                         (object ?object)
                         (target (desig:a location (pose ?pose))))))

(defun test-switch-two-bottles ()
  (spawn-two-bottles)
  (proj:with-projection-environment urdf-proj:urdf-bullet-projection-environment
    (cpl:top-level
      ;; Go to counter top and perceive bottle
      (let ((?navigation-goal *pose-counter*)
            (?ptu-goal 
              (cl-transforms-stamped:make-pose-stamped
               "base_footprint"
               0.0
               (cl-transforms:make-3d-vector 0.65335d0 0.076d0 0.758d0)
               (cl-transforms:make-identity-rotation))))
        (cpl:par
          ;; Move torso up
          (exe:perform
           (desig:a motion (type moving-torso) (joint-angle 0.3)))
          (exe:perform
           (desig:an action
                     (type parking-arms)))
          (navigate-to ?navigation-goal))
        (look-at ?ptu-goal))
      ;; Pick up bottle-1 with right arm.
      (let ((?perceived-bottle-1 (get-perceived-bottle-desig)))
        (pick-up ?perceived-bottle-1 :right)
        (exe:perform
         (desig:an action
                   (type parking-arms)
                   (arms (:right))))
        ;; Move to the meal table
        (let ((?pose *pose-meal-table*))
          (navigate-to ?pose))
        ;; Pick up bottle-2 with left arm
        (let ((?perceived-bottle-2 (get-perceived-bottle-desig)))
          (pick-up ?perceived-bottle-2 :left)
          ;; Move left arm out of sight
          (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (:left))))
          ;; Place bottle-1 on second table
          (let ((?drop-pose *pose-bottle-2*))
            (place-down ?drop-pose ?perceived-bottle-1 :right))
          ;; Move right arm out of sight
          (exe:perform
           (desig:an action
                     (type parking-arms)
                     (arms (:right))))
          ;; Move to the counter table
          (let ((?navigation-goal *pose-counter*))
            (navigate-to ?navigation-goal))
          ;; Place bottle-2 on the counter
          (let ((?drop-pose *pose-bottle-1*))
            (place-down ?drop-pose ?perceived-bottle-2 :left))
          (exe:perform
           (desig:an action
                     (type parking-arms))))))))

MAKE-POSE

VISUALIZE-COORDINATES

MOVE-KITCHEN-JOINT

ADD-OBJECTS-TO-MESH-LIST

SPAWN-OBJECT

...

LOOK-AT

GET-PERCEIVED-BOTTLE-DESIG

PICK-UP

PLACE-DOWN

TEST-SWITCH-TWO-BOTTLES